使用 pandas 的代码。

问题 5：建立机器学习算法模型，预测未来航班取消情况。并对模型预测情况进行评估。

下面是三种方法：

1. k_means 聚类算法。
2. random_forest 随机森林算法。
3. lstm 长短期记忆网络算法。

In [ ]:

# k_means
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt


# 读取 csv 格式数据集到变量 df
df = pd.read_csv("2008.csv", index_col=0)
# 输出 df 的行名与列名，以验证是否读取正确
df.index
df.columns
# 数据准备
data_X = df[['Month', 'DayofMonth', 'DayOfWeek',
             'UniqueCarrier', 'Origin', 'Dest', 'Distance']]

# 处理分类特征：将字符串编码为数值
label_encoders = {}
categorical_columns = ['UniqueCarrier', 'Origin', 'Dest']
for col in categorical_columns:
    le = LabelEncoder()
    data_X[col] = le.fit_transform(data_X[col])
    label_encoders[col] = le

# 数据标准化
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data_X)

# 使用 K-means 进行聚类
k = 2  # 假设聚类为 2 类（Cancelled 和未 Cancelled）
kmeans = KMeans(n_clusters=k, random_state=42)
clusters = kmeans.fit_predict(data_scaled)

# 将聚类结果添加回原数据
df['Cluster'] = clusters

# 查看聚类结果分布
print("聚类结果统计：")
print(df.groupby('Cluster')['Cancelled'].value_counts())


# 使用 PCA 降维到 2 维
pca = PCA(n_components=2)
data_pca = pca.fit_transform(data_scaled)

# 计算轮廓系数
silhouette_avg = silhouette_score(data_pca, clusters)
print(f"Silhouette Score (PCA Reduced Data): {silhouette_avg:.2f}")


# 可视化聚类中心
plt.figure(figsize=(8, 6))
plt.scatter(data_scaled[:, 0], data_scaled[:, 1],
            c=clusters, cmap='viridis', s=10)
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[
            :, 1], c='red', marker='x', s=200)
plt.title("K-means Clustering Results")
plt.xlabel("Feature 1")
plt.ylabel("Feature 2")
plt.show()

In [ ]:

# random_forest
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


# 读取 csv 格式数据集到变量 df
df = pd.read_csv("2008.csv", index_col=0)
# 输出 df 的行名与列名，以验证是否读取正确
df.index
df.columns

# 数据准备
data_X = df[['Month', 'DayofMonth', 'DayOfWeek',
             'UniqueCarrier', 'Origin', 'Dest', 'Distance']]
data_Y = df['Cancelled']

# 处理分类特征：将字符串编码为数值
label_encoders = {}
categorical_columns = ['UniqueCarrier', 'Origin', 'Dest']
for col in categorical_columns:
    le = LabelEncoder()
    data_X[col] = le.fit_transform(data_X[col])
    label_encoders[col] = le

# 数据集划分
X_train, X_test, Y_train, Y_test = train_test_split(
    data_X, data_Y, test_size=0.2, random_state=42)

# 构建随机森林分类器
model = RandomForestClassifier(n_estimators=100, random_state=42)

# 训练模型
model.fit(X_train, Y_train)

# 预测结果
Y_pred = model.predict(X_test)

# 评估模型
accuracy = accuracy_score(Y_test, Y_pred)
print(f"Test Accuracy: {accuracy:.2f}")
print("Classification Report:")
print(classification_report(Y_test, Y_pred))
print("Confusion Matrix:")
print(confusion_matrix(Y_test, Y_pred))

In [ ]:

# lstm
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense


# 读取 csv 格式数据集
df = pd.read_csv("2008.csv", index_col=0)

# 对分类变量进行编码（因为只能输入数字数据，所以需要进行编码）
label_encoder = LabelEncoder()
df['UniqueCarrier'] = label_encoder.fit_transform(df['UniqueCarrier'])
df['Origin'] = label_encoder.fit_transform(df['Origin'])
df['Dest'] = label_encoder.fit_transform(df['Dest'])

# 输出 df 的行名与列名，以验证是否读取正确
print(df.columns)

# 数据准备
data_X = df[['Month', 'DayofMonth', 'DayOfWeek',
             'UniqueCarrier', 'Origin', 'Dest', 'Distance']]
data_Y = df['Cancelled']

# 数据归一化
scaler = MinMaxScaler()
data_X_scaled = scaler.fit_transform(data_X)

# 数据重塑
time_steps = 1  # 假设每个时间步是 1
X = np.reshape(
    data_X_scaled, (data_X_scaled.shape[0], time_steps, data_X_scaled.shape[1]))

# 数据集划分
X_train, X_test, Y_train, Y_test = train_test_split(
    X, data_Y, test_size=0.2, random_state=42)

# 构建 LSTM 模型
model = Sequential()
model.add(LSTM(50, return_sequences=False,
          input_shape=(time_steps, X.shape[2])))
model.add(Dense(1, activation='sigmoid'))  # 用于分类任务
model.compile(optimizer='adam', loss='binary_crossentropy',
              metrics=['accuracy'])

# 训练模型
model.fit(X_train, Y_train, epochs=2, batch_size=32,
          validation_data=(X_test, Y_test))

# 评估模型
loss, accuracy = model.evaluate(X_test, Y_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

# 预测测试集
Y_pred = model.predict(X_test)
Y_pred_classes = (Y_pred > 0.5).astype(int)  # 将概率转换为 0/1 分类

# 生成混淆矩阵
cm = confusion_matrix(Y_test, Y_pred_classes)

# 显示混淆矩阵
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=[
            'Not Cancelled', 'Cancelled'], yticklabels=['Not Cancelled', 'Cancelled'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

# 输出分类报告
print("Classification Report:")
print(classification_report(Y_test, Y_pred_classes))
# 预测测试集标签 (输出概率)
Y_pred = model.predict(X_test)

# 将概率转换为 0/1 二值标签
Y_pred_classes = (Y_pred > 0.5).astype(int)

# 生成混淆矩阵
print("Confusion Matrix:")
print(confusion_matrix(Y_test, Y_pred_classes))